# Ensemble Learning

## Initial Imports

In [83]:
import warnings
warnings.filterwarnings('ignore')

In [84]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [85]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [86]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [87]:
# Create our features
df = df.copy()
drop = pd.get_dummies(df, columns=["home_ownership", "verification_status", "application_type", "pymnt_plan", "initial_list_status", "hardship_flag", "debt_settlement_flag"])
X = drop.drop(columns=["loan_status", "issue_d", "next_pymnt_d"])

# Create our target
y = pd.DataFrame(df["loan_status"])

In [88]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App,pymnt_plan_n,initial_list_status_f,initial_list_status_w,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.478007,0.373992,0.148001,0.860340,0.139660,1.0,0.123879,0.876121,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.499520,0.483865,0.355104,0.346637,0.346637,0.0,0.329446,0.329446,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,1.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,1.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.000000,0.000000,1.000000,0.000000,1.0,0.000000,1.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.0


In [89]:
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App,pymnt_plan_n,initial_list_status_f,initial_list_status_w,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,1,0,1,0,1,0,1,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,0,0,1,1,0,1,0,1,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,0,0,1,1,0,1,0,1,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0,0,1,1,0,1,0,1,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,0,0,1,0,1,0,1,1,1


In [90]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk       68470
high_risk        347
dtype: int64

In [91]:
# Split the X and y into X_train, X_test, y_train, y_test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [92]:
# Create the StandardScaler instance
data_scaler = StandardScaler()

In [93]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [94]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [95]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [102]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_rf = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred_rf)

0.732575308910564

In [99]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = brf.predict(X_test)
confusion_matrix(y_test, y_pred_rf)

array([[   52,    35],
       [ 2269, 14849]], dtype=int64)

In [103]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.60      0.87      0.04      0.72      0.50        87
   low_risk       1.00      0.87      0.60      0.93      0.72      0.53     17118

avg / total       0.99      0.87      0.60      0.92      0.72      0.53     17205



In [105]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.0703323153279889, 'total_rec_prncp'),
 (0.06525204322950455, 'total_rec_int'),
 (0.061624248400504525, 'last_pymnt_amnt'),
 (0.05448360982760552, 'total_pymnt_inv'),
 (0.05290301389123787, 'total_pymnt'),
 (0.029547244150478137, 'int_rate'),
 (0.020380096635883443, 'dti'),
 (0.018166231652089665, 'annual_inc'),
 (0.01797914658526676, 'max_bal_bc'),
 (0.01710595338451746, 'bc_util'),
 (0.01706853845142887, 'mo_sin_old_il_acct'),
 (0.01691251180600299, 'installment'),
 (0.01671903423212854, 'mths_since_recent_inq'),
 (0.01638582141267288, 'il_util'),
 (0.016234193003927918, 'avg_cur_bal'),
 (0.015894844105566214, 'mo_sin_old_rev_tl_op'),
 (0.015844620015384576, 'out_prncp_inv'),
 (0.015752223477820654, 'out_prncp'),
 (0.01565942863911724, 'revol_bal'),
 (0.015084770360533965, 'mths_since_rcnt_il'),
 (0.014603009219444544, 'all_util'),
 (0.014468998763037947, 'total_bc_limit'),
 (0.014149477484153091, 'tot_cur_bal'),
 (0.014053500003188267, 'tot_hi_cred_lim'),
 (0.013838136727127746, 

### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=1000, random_state=1)
eec.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_eec = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred_eec)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = eec.predict(X_test)
confusion_matrix(y_test, y_pred_eec)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_eec))

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.